In [107]:
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 25 22:20:00 2019

@author: byjiang
"""

'''
easiest pushing box

0: boundary and wall
1: path can be moved
2: player
3: box
4: storage



Known
[[0 0 0 0 0 0]
 [0 4 3 1 2 0]
 [0 0 0 0 0 0]]
Transform               ## Remove storage
[[0 0 0 0 0 0]
 [0 1 3 1 2 0
 [0 0 0 0 0 0]]
result                 ## Only show storage 
                          when coordinates of boxes match coordinates of storage: complete
[[0 0 0 0 0 0]          
 [0 4 1 1 1 0]
 [0 0 0 0 0 0]]

'''

'\neasiest pushing box\n\n0: boundary and wall\n1: path can be moved\n2: player\n3: box\n4: storage\n5: box is put in the position of storage (box+storage)\n6: person is put in the position of storage (person+storage)\n\n\nKnown\n[[0 0 0 0 0 0]\n [0 4 3 1 2 0]\n [0 0 0 0 0 0]]\n\nresult\n[[0 0 0 0 0 0]\n [0 5 2 1 1 0]\n [0 0 0 0 0 0]]\n\n'

In [274]:
import numpy as np
import copy
def person_position(P_1):
    
    # search the position of player
    
    for i in range(len(P_1)): #length
        for j in range(len(P_1[0])): #width
            if P_1[i][j] == 2 or P_1[i][j] == 6:
                return ([i,j])

def box_position(P_1):
    
    # search the position of box
    box = []
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j] == 3:
                box.append([i,j])
    return (box)


def target_position(P_1):
    
    # search the position of box
    target = []
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j] == 4:
                target.append([i,j])
    return (target)

def transform_P1(P):
    ## initial state, ignore storage
    P_1=copy.deepcopy(P)
    for i in range(len(P)):
        for j in range(len(P[0])):
            if P_1[i][j]==4:
                P_1[i][j]=1
    return P_1

def transform_Pend(P):
    ## final state，ignore person and boxes all in storage
    P_end=copy.deepcopy(P)
    for i in range(len(P)):
        for j in range(len(P[0])):
            if P_end[i][j]==2 or P_end[i][j]==3:
                P_end[i][j]=1
    return P_end
def arrayin(P1,P2):
    for i in range(len(P2)):
        if (P1-P2[i]).any():
            continue
        else:
            return True
    return False


############################################################我的改动
dirs=[[0,-1],[-1,0],[0,1],[1,0]]## four directions 'L''U''R''D'
def is_OK(P_1,P_end):
    ## check if current state match final state
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j]!=3 and  P_end[i][j]==4:
                return False
    return True

In [275]:
visited=np.array([P_1])## record visited states
states=[]## frontier, record states to explore
pathfinal=[]## record final path
iterate_max=100## death loop maximum
def BFS(P_1,person):
    states=[[P_1,[],person]]
    while(len(states)>0):
        global pathfinal
        P=np.array(states[0][0])##initial status
        path=states[0][1]
        person=states[0][2] 
        if(len(path)>iterate_max):                                 ##deal with too long loop
            break
        if(is_OK(P,P_end)):
            if len(pathfinal)==0:
                pathfinal=path
                continue                                                 ##check if ok
            elif len(path)<len(pathfinal):
                pathfinal=path
                continue
        states=states[1:]  ##explore the current status,update the frontier
        for i,dir in enumerate(dirs):
            neg=[person[0]+dir[0],person[1]+dir[1]]##the neighbour
            nneg=[person[0]+2*dir[0],person[1]+2*dir[1]]   ##the neighbour of the neighbour
            if not (neg[0]>=0 and neg[0]<len(P) and neg[1]>=0 and neg[1]<len(P[0])):
                continue
            if P[neg[0]][neg[1]]==1:           ## only the person move
                global visited
                Pnew=copy.deepcopy(P)
                Pnew[neg[0]][neg[1]]=2
                Pnew[person[0]][person[1]]=1
                if not arrayin(Pnew,visited):## check if visited
                    visitedlist=visited.tolist()
                    visitedlist.append(Pnew.tolist())
                    visited=np.array(visitedlist)
                    states.append([Pnew,path+[dir],neg])  ##add new status in frontier
            elif P[person[0]][person[1]]==2 and P[neg[0]][neg[1]]==3 and P_end[nneg[0]][nneg[1]]!=0:
                Pnew=P
                Pnew[neg[0]][neg[1]]=2
                Pnew[person[0]][person[1]]=1
                Pnew[nneg[0]][nneg[1]]=3
                if not arrayin(Pnew,visited):                                            ## check if visited
                    visitedlist=visited.tolist()
                    visitedlist.append(Pnew.tolist())
                    visited=np.array(visitedlist)
                    states.append([Pnew,path+[dir],neg])                           ##add new status in frontier

In [277]:
P = np.array([[0,0,0,0,0,0,0],[0,1,1,1,1,2,0],[0,4,3,1,3,4,0],[0,0,0,0,0,0,0]])
person=person_position(P)
print(person)
P_1=transform_P1(P)
print(P_1)
P_end=transform_Pend(P)
print(P_end)
BFS(P_1,person)
moves=[]
for i in range(len(pathfinal)):
    if pathfinal[i]==[0,-1]:
        moves.append('L')
    if pathfinal[i]==[-1,0]:
        moves.append('U')
    if pathfinal[i]==[0,1]:
        moves.append('R')
    if pathfinal[i]==[1,0]:
        moves.append('D')
print(moves)

[1, 5]
[[0 0 0 0 0 0 0]
 [0 1 1 1 1 2 0]
 [0 1 3 1 3 1 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 1 1 1 1 1 0]
 [0 4 1 1 1 4 0]
 [0 0 0 0 0 0 0]]
['L', 'L', 'D', 'L', 'R', 'R']
